# Part 1

First, I want to break the flattened input string into layers.

In [1]:
import numpy as np

def separate_layers(image_str, width, height):
    """Returns the image as np.ndarray[layer, row, column]"""
    flat = np.fromiter((int(s) for s in image_str), dtype=int)
    # Reshape order is layers, rows, columns
    layers = flat.reshape((-1, height, width))
    return layers

Check if this is working right. First index of the returned array should be the layer number, second should be row, third should be column.

In [2]:
example_str = '123456789012'
layers = separate_layers(example_str, width=3, height=2)
print(layers)
print(layers[0])    # [[1 2 3] [4 5 6]]
print(layers[0,0])  # [1 2 3]
print(layers[1,1])  # [0 1 2]

[[[1 2 3]
  [4 5 6]]

 [[7 8 9]
  [0 1 2]]]
[[1 2 3]
 [4 5 6]]
[1 2 3]
[0 1 2]


`np.unique` will provide counts of all digits in the layers, so that buys me both the 0 counts and the 1 and 2 counts. So, make one function to get the digit counts for each layer, then another to check the 0 counts so I can pick the right layer.

In [3]:
def digit_counts(layer):
    """For a layer, returns {value1:count, value2:count, ...}"""
    values, counts = np.unique(layer, return_counts=True)
    return dict(zip(values, counts))

In [4]:
def zeros_and_counts(layers):
    """Return (num zeros, counts dict) for each layer in layers."""
    counts = []
    for layer in layers[:]:
        count = digit_counts(layer)
        try:
            zeros = count[0]
        except KeyError:
            # Layer has no 0
            zeros = 0
        counts.append((zeros, count))
    return counts

Now pull the elements together for part 1. Split up the layers, get the zero counts and digit counts, find the layer with the fewest zeros, then multiply the count of 1s by the count of 2s in that layer.

In [5]:
def part1(input_str, width, height):
    layers = separate_layers(input_str, width, height)
    counts = zeros_and_counts(layers)
    # Sort low to high by number of 0s
    counts.sort(key=lambda x:x[0])
    # Grab the count dict for the layer with the fewest zeros (the first element in counts), then
    # multiply the # of 1s by the # of 2s
    layer_counts = counts[0][1]
    return layer_counts[1]*layer_counts[2]

For the example, layer 1 has the fewest 0s (0), and has one 1 and one 2, so the answer will be 1.

In [6]:
print(part1(example_str, 3, 2))

1


Now run the problem input.

In [7]:
with open('day8_input.txt', 'r') as infile:
    input_str = infile.read().strip()
print(part1(input_str, width=25, height=6))

2159


# Part 2

For this part, for each pixel in the image I need to iterate through the layers and return the first occurence of either 0 or 1. 2s get passed over since they are transparent.

First, construct the example.

In [8]:
example_str = '0222112222120000'
ex_layers = separate_layers(example_str, width=2, height=2)
print(ex_layers)

[[[0 2]
  [2 2]]

 [[1 1]
  [2 2]]

 [[2 2]
  [1 2]]

 [[0 0]
  [0 0]]]


In [9]:
def decode_image(layers):
    # Initialize the single-layer decoded image
    decoded_image = np.empty(tuple(layers.shape[1:]), dtype=int)
    
    # Look down the layers for each pixel, find the first non-2 digit and put it in the output
    for row in range(layers.shape[1]):
        for col in range(layers.shape[2]):
            layer = layers[:, row, col]
            for v in layer:
                if v in [0, 1]:
                    decoded_image[row, col] = v
                    break
    return decoded_image

Test with the example. Output should be:

0 1<br>
1 0

In [10]:
print(decode_image(ex_layers))

[[0 1]
 [1 0]]


Now the puzzle input:

In [11]:
layers = separate_layers(input_str, width=25, height=6)
print(decode_image(layers))

[[0 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 0]
 [1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0]
 [1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 1 0 0 1 0]
 [1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 0]
 [1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0 0]
 [0 1 1 0 0 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0]]


Looks like: C J Z H R